# Gridsearch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
print("Python version")
print(sys.version)


Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
import sklearn
print("scikit-learn version")
print(sklearn.__version__)


scikit-learn version
1.2.2


In [ ]:
df_path = '/content/drive/MyDrive/모델 튜닝/train.csv'
df = pd.read_csv(df_path)

In [ ]:
X = df.drop(['person_id', 'login'], axis=1)
y = df['login']

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# 하이퍼파라미터 검색 그리드 설정
param_grid = {
    'n_estimators': [200, 400, 600, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 50, 100],
    'min_impurity_decrease': [0.0, 0.1, 0.2],
    'bootstrap': [True, False]
}

# Random Forest 분류기 인스턴스 생성
rf = RandomForestClassifier()

cv = StratifiedKFold(n_splits=5)

# GridSearchCV 인스턴스 생성
grid_search = RandomSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring='roc_auc', verbose=2, n_jobs=-1)

# GridSearchCV를 사용한 하이퍼파라미터 최적화 실행
grid_search.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Parameters: {'n_estimators': 984, 'min_weight_fraction_leaf': 0.0281, 'min_samples_split': 4, 'min_samples_leaf': 6, 'min_impurity_decrease': 4e-05, 'max_leaf_nodes': 81, 'max_features': 'auto', 'max_depth': 9, 'criterion': 'gini', 'bootstrap': True}
Best Score: 0.9182621157614577


In [ ]:
# submission
submit = pd.read_csv('/content/drive/MyDrive/모델 튜닝/sample_submission.csv')

# GridSearchCV로 찾은 최적의 파라미터
best_params = grid_search.best_params_

# 찾은 최적의 파라미터들을 제출 양식에 맞게 업데이트
for param, value in best_params.items():
    if param in submit.columns:
        submit.loc[0, param] = value

# 제출 파일 저장 경로
submit_file_path = 'submit.csv'
submit.to_csv(submit_file_path, index=False)

# 제출 파일 다운로드
from google.colab import files
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>